In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from fastai import *
from fastai.vision import *

## LSun data

In [ ]:
#path = Path('data/lsun')
#path_img = path/'bedroom'
path = untar_data(URLs.PETS)

In [ ]:
train_ds = (ImageItemList.from_folder(path).label_const(0., label_cls=FloatList)
               .transform(tfms=[crop_pad(size=64, row_pct=(0,1), col_pct=(0,1))],size=64))

In [ ]:
data = (ImageDataBunch.create(train_ds, valid_ds=None, path=path)
                     .normalize(stats = [torch.tensor([0.5,0.5,0.5]), torch.tensor([0.5,0.5,0.5])]))

## Models

In [ ]:
wgan = models.BasicGAN(64, 100, 3, 64, 1)
learn = Learner(data, wgan, loss_func=NoopLoss(), opt_func=optim.RMSprop, wd=0.)

In [ ]:
cb = callbacks.GANTrainer(learn, bs=128, n_disc_iter=callbacks.first_disc_iter)
learn.callbacks.append(cb)
learn.fit(1, 1e-4)

In [ ]:
learn.recorder.losses[:10]

In [ ]:
cb.dlosses[:10]

In [ ]:
%debug

In [ ]:
x,y = next(iter(learn.data.train_dl))

In [ ]:
tst = learn.model(callbacks.create_noise(x,64,100,False), gen=True)
img = data.denorm(tst[0].cpu()).numpy().clip(0,1).transpose(1,2,0)
plt.imshow(img)

In [ ]:
learn.model(tst)

In [ ]:
learn.model(x)